In [2]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import itertools
#from tqdm import tqdm
from sklearn.model_selection import cross_val_score, KFold

In [3]:
df = pd.read_csv('logP.csv')
y = df.logP
X = df.iloc[:, 3:]
feats = X.columns
df

,Unnamed: 0,smiles,logP,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,COc1ccccc1[C@@H]1CC(=O)Nc2cc(C)c(C)cc21,4.17,0.909,12.053455,12.053455,0.045556,0.045556,0.909253,281.355,...,0,0,0,0,0,0,0,0,0,0
1,1,COc1ccc2c(c1)O[C@](O)(C(F)(F)F)CC2=O,2.79,0.839,12.616874,12.616874,0.009514,-5.052156,0.839542,262.183,...,0,0,0,0,0,0,0,0,0,0
2,2,C=C(c1ccc(N(C)C)cc1)c1ccc(N(C)C)cc1,5.30,0.828,4.217020,4.217020,1.056122,1.056122,0.828333,266.388,...,0,0,0,0,0,0,0,0,0,0
3,3,Cc1ccc([C@@H](O)C#CC(O)(c2ccccc2)c2ccccc2)o1,3.53,0.726,11.239605,11.239605,0.364327,-1.508792,0.726080,318.372,...,0,0,0,0,0,0,0,0,0,0
4,4,CC(C)(C)c1cc(Cc2ccccc2)n[nH]1,4.35,0.816,4.352931,4.352931,0.140811,0.140811,0.816043,214.312,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,239,Cc1cccc(NC(=O)C[C@@H]2S/C(=N\c3c(C)cccc3C)N(C)...,4.75,0.863,12.562081,12.562081,0.093167,-0.455663,0.863227,381.501,...,1,0,0,0,0,0,0,0,0,0
240,240,CCOc1ccccc1OCC(=O)Nc1ccc2ccccc2c1,4.73,0.737,12.114099,12.114099,0.072506,-0.211498,0.737878,321.376,...,0,0,0,0,0,0,0,0,0,0
241,241,OCC#CC(O)(c1ccc(F)cc1)c1ccc(F)cc1,3.61,0.824,12.974979,12.974979,0.339313,-1.732014,0.824192,274.266,...,0,0,0,0,0,0,0,0,0,0
242,242,CC(=O)Nc1ccc(/N=C/c2c([O-])oc3ccccc3c2=O)cc1,3.64,0.750,12.347197,12.347197,0.125336,-0.724707,0.750035,321.312,...,0,0,0,0,0,0,0,0,0,0


In [4]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

## L1

### without cv in L1

In [5]:
model_l1 = linear_model.Lasso()
model_l1.fit(X, y)
r2_L1 = model_l1.score(X, y)
r2_L1

0.5064268989978196

In [6]:
selected_features_l1_Bool = model_l1.coef_ != 0
X_new_l1_names = X.columns[selected_features_l1_Bool]
X_new_l1 = X[X_new_l1_names]
X_new_l1

,MolWt,ExactMolWt,BCUT2D_MWHI,BertzCT,Ipc,PEOE_VSA6,PEOE_VSA7,SMR_VSA5,SlogP_VSA2,SlogP_VSA5,TPSA,EState_VSA8
0,281.355,281.141579,16.465851,706.359669,6.975974e+04,24.265468,42.670563,26.186202,13.016977,34.592534,38.33,31.296997
1,262.183,262.045293,19.413883,497.551959,7.977097e+03,0.000000,12.132734,18.384231,29.962980,16.778810,55.76,4.736863
2,266.388,266.178299,15.059973,524.320217,3.216838e+04,30.844404,40.965476,0.000000,28.190688,11.126903,6.48,64.909692
3,318.372,318.125594,16.364915,814.604445,4.328251e+05,72.504539,19.056471,18.628754,10.213055,28.751364,53.60,11.840869
4,214.312,214.146999,15.120847,449.087623,5.439498e+03,51.103047,11.629819,32.607024,10.197364,37.722519,28.68,61.300411
...,...,...,...,...,...,...,...,...,...,...,...,...
239,381.501,381.151098,32.166721,903.142781,1.118114e+06,42.093720,49.594300,32.441971,34.179531,23.111176,61.77,5.316789
240,321.376,321.136493,16.521215,844.916502,3.832465e+05,42.464569,41.961654,6.923737,19.120944,6.923737,47.56,5.316789
241,274.266,274.080536,19.143514,591.417357,3.332934e+04,36.106337,24.265468,5.601051,16.819937,11.126903,40.46,11.840869
242,321.312,321.088080,16.471022,988.066548,2.911067e+05,12.132734,36.398202,6.923737,12.121780,12.487189,94.73,10.309193


In [31]:
model1 = linear_model.LinearRegression()
coeff1 = cross_val_score(model1, X_new_l1 , y, cv = kf).mean()
coeff1

0.47136805155291583

### with cv in L1

In [7]:
model_l1_cv = linear_model.LassoCV(alphas = [0.1] , cv = kf)
model_l1_cv.fit(X, y)
r2_L1_cv = model_l1_cv.score(X, y)
r2_L1_cv

/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.282e+01, tolerance: 1.813e-02
  model = cd_fast.enet_coordinate_descent(
/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.931e+01, tolerance: 1.692e-02
  model = cd_fast.enet_coordinate_descent(
/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.88

0.634127607626456

In [8]:
selected_features_l1_Bool_cv = model_l1_cv.coef_ != 0
X_new_l1_names_cv = X.columns[selected_features_l1_Bool_cv]
X_new_l1_cv = X[X_new_l1_names_cv]
X_new_l1_cv

,NumValenceElectrons,BCUT2D_MWHI,Ipc,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA14,PEOE_VSA2,PEOE_VSA4,...,EState_VSA1,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState2
0,108,16.465851,6.975974e+04,124.497248,10.053652,5.749512,0.000000,0.00000,4.794537,0.000000,...,0.000000,11.825086,6.420822,17.000350,16.690354,24.265468,31.296997,4.736863,5.471081,12.053455
1,98,19.413883,7.977097e+03,99.929735,14.580253,11.499024,5.783245,11.96341,4.794537,13.171245,...,24.167476,17.062475,0.000000,6.066367,19.242532,0.000000,4.736863,4.736863,47.192212,11.566614
2,104,15.059973,3.216838e+04,121.508214,9.799819,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,16.700008,11.374773,28.190688,64.909692,0.000000,0.000000,4.191308
3,120,16.364915,4.328251e+05,140.752324,14.630206,11.520495,11.705017,0.00000,0.000000,0.000000,...,11.705017,0.000000,22.647398,0.000000,0.000000,60.663671,11.840869,4.417151,5.387791,0.000000
4,84,15.120847,5.439498e+03,96.710470,0.000000,0.000000,0.000000,0.00000,5.098682,5.098682,...,0.000000,5.414990,0.000000,12.114750,11.257379,6.066367,61.300411,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,140,32.166721,1.118114e+06,163.418318,5.316789,5.249938,5.167652,0.00000,14.488984,0.000000,...,5.249938,18.235181,5.167652,28.065127,11.761885,63.235781,5.316789,4.992405,0.000000,31.169713
240,122,16.521215,3.832465e+05,140.568405,14.790515,0.000000,18.105906,0.00000,4.794537,0.000000,...,0.000000,12.514062,18.105906,16.459835,0.000000,67.587408,5.316789,9.473726,11.053188,12.114099
241,102,19.143514,3.332934e+04,114.867029,10.213055,18.241324,5.601051,0.00000,0.000000,0.000000,...,23.842374,0.000000,11.126903,0.000000,48.530937,0.000000,11.840869,5.106527,25.949958,0.000000
242,120,16.471022,2.911067e+05,136.632268,14.840467,0.000000,5.428790,0.00000,14.581479,0.000000,...,11.375098,17.053651,16.760997,0.000000,13.138338,0.000000,10.309193,4.417151,5.130607,27.445004


In [42]:
model1_cv = linear_model.LinearRegression()
coeff1_cv = cross_val_score(model1_cv, X_new_l1_cv , y, cv = kf).mean()
coeff1_cv

0.47136805155291583

## l2

### L2 without cv in l2

In [9]:
model_l2 = linear_model.Ridge(alpha=0.5)
model_l2.fit(X, y)
r2_L2 = model_l2.score(X, y)
r2_L2

/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=2.04289e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


0.8334953518204348

In [62]:
selected_features_l2_Bool = model_l2.coef_ != 0
X_new_l2_names = X.columns[selected_features_l2_Bool]
X_new_l2 = X[X_new_l2_names]
X_new_l2

,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_priamide,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.909,12.053455,12.053455,0.045556,0.045556,0.909253,281.355,262.203,281.141579,108,...,0,0,0,0,0,0,0,0,0,0
1,0.839,12.616874,12.616874,0.009514,-5.052156,0.839542,262.183,253.111,262.045293,98,...,0,0,0,0,0,0,0,0,0,0
2,0.828,4.217020,4.217020,1.056122,1.056122,0.828333,266.388,244.212,266.178299,104,...,0,0,0,0,0,0,0,0,0,0
3,0.726,11.239605,11.239605,0.364327,-1.508792,0.726080,318.372,300.228,318.125594,120,...,0,0,0,0,0,0,0,0,0,0
4,0.816,4.352931,4.352931,0.140811,0.140811,0.816043,214.312,196.168,214.146999,84,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.863,12.562081,12.562081,0.093167,-0.455663,0.863227,381.501,358.317,381.151098,140,...,0,0,1,0,0,0,0,0,0,0
240,0.737,12.114099,12.114099,0.072506,-0.211498,0.737878,321.376,302.224,321.136493,122,...,0,0,0,0,0,0,0,0,0,0
241,0.824,12.974979,12.974979,0.339313,-1.732014,0.824192,274.266,262.170,274.080536,102,...,0,0,0,0,0,0,0,0,0,0
242,0.750,12.347197,12.347197,0.125336,-0.724707,0.750035,321.312,308.208,321.088080,120,...,0,0,0,0,0,0,0,0,0,0


In [59]:
model2 = linear_model.LinearRegression()
coeff2 = cross_val_score(model2, X_new_l2 , y, cv = kf)
max(coeff2)

-1.157175965072533

### with cv

In [13]:
model_l2_cv = linear_model.RidgeCV(cv = kf)
model_l2_cv.fit(X, y)
r2_L2_cv = model_l2_cv.score(X, y)
r2_L2_cv

/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.local/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.local/lib/python3.11/site-pa

0.7688197313610731

In [11]:
model_l2_cv = linear_model.RidgeCV(cv = kf)
model_l2_cv.fit(X, y)
score = cross_val_score(model_l2_cv(), X_new, y, cv = kf)
score

/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.venv/lib64/python3.11/site-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/stas/.venv/lib64/python3.11/s

TypeError: 'RidgeCV' object is not callable

In [14]:
selected_features_l2_Bool_cv = model_l2_cv.coef_ != 0
X_new_l2_names_cv = X.columns[selected_features_l2_Bool_cv]
X_new_l2_cv = X[X_new_l2_names_cv]
X_new_l2_cv

,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_priamide,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.909,12.053455,12.053455,0.045556,0.045556,0.909253,281.355,262.203,281.141579,108,...,0,0,0,0,0,0,0,0,0,0
1,0.839,12.616874,12.616874,0.009514,-5.052156,0.839542,262.183,253.111,262.045293,98,...,0,0,0,0,0,0,0,0,0,0
2,0.828,4.217020,4.217020,1.056122,1.056122,0.828333,266.388,244.212,266.178299,104,...,0,0,0,0,0,0,0,0,0,0
3,0.726,11.239605,11.239605,0.364327,-1.508792,0.726080,318.372,300.228,318.125594,120,...,0,0,0,0,0,0,0,0,0,0
4,0.816,4.352931,4.352931,0.140811,0.140811,0.816043,214.312,196.168,214.146999,84,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.863,12.562081,12.562081,0.093167,-0.455663,0.863227,381.501,358.317,381.151098,140,...,0,0,1,0,0,0,0,0,0,0
240,0.737,12.114099,12.114099,0.072506,-0.211498,0.737878,321.376,302.224,321.136493,122,...,0,0,0,0,0,0,0,0,0,0
241,0.824,12.974979,12.974979,0.339313,-1.732014,0.824192,274.266,262.170,274.080536,102,...,0,0,0,0,0,0,0,0,0,0
242,0.750,12.347197,12.347197,0.125336,-0.724707,0.750035,321.312,308.208,321.088080,120,...,0,0,0,0,0,0,0,0,0,0


In [52]:
model2_cv = linear_model.LinearRegression()
coeff2_cv = cross_val_score(model2_cv, X_new_l2_cv , y, cv = kf)
max(coeff2_cv)

-1.15717589036656